# **Dataset tutorial: A large and rich EEG dataset for modeling human visual object recognition**

In this tutorial you will load, visualize and model the preprocessed EEG data, the corresponding image conditions and the deep neural network (DNN) feature maps of these image conditions from the data paper [A large and rich EEG dataset for modeling human visual object recognition][paper_link]. This dataset contains the EEG responses of 10 participants to 16,540 training image conditions (with 4 repeats per condition) and 200 test image conditions (with 80 repeats per condition). [This video][rsvp] shows you the rapid serial visual presentation (RSVP) paradigm used to collect the EEG dataset. The largness of this dataset enables the modeling of EEG data through several machine learning and deep learning methods, including building end-to-end encoding models of EEG responses to arbitrary images starting from randomly initialized DNNs. The complete dataset (in its raw and preprocessed format) along with the stimuli images, DNN feature maps and detailed data descriptions are available on [OSF][osf]. Please refer to the [paper][paper_link] for information regarding stimuli images, experimental paradigm, EEG acquisition/preprocessing, DNN feature maps extraction and dataset validation through several computational modeling approaches. All the paper results can be reproduces through the code available on [GitHub][git].

The data used in this tutorial is found in a Google Drive public folder called [```tutorial_data```][data]. Before running the tutorial code you need to select this folder and choose "Add a shortcut to Drive". This will create a shortcut (without copying or taking space) of the folder to a desired path in your Google Drive, from which you can read the content after mounting using ```drive.mount()```.

[paper_link]: https://www.biorxiv.org/content/10.1101/2022.03.15.484473v1
[rsvp]: https://youtu.be/JhpvpHlfPlE
[osf]: https://osf.io/3jk45/
[git]: https://github.com/gifale95/eeg_encoding
[data]: https://drive.google.com/drive/folders/1ZejP4F9Isj2A_QSlxbPzvOP16-BdBerk?usp=sharing

## 1. Import libraries and mount Google Drive to access the data

To start, you will import the Python libraries used throughout the tutorial and mount your Google Drive to access the data. Make sure that you edit the ```gdrive_data_parent_dir``` variable with the path to the ```tutorial_data``` shortcut folder you added to your Drive.



In [ ]:
from google.colab import drive
import os
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr as corr

drive.mount('/content/gdrive/', force_remount=True)
gdrive_data_parent_dir = '/content/gdrive/MyDrive/.../tutorial_data' #@param {type:"string"}

## 2. Load the preprocessed EEG data

Now you will load participant's 1 preprocessed EEG data. The EEG data consists of two files:
* ```preprocessed_eeg_training.npy```: the preprocessed EEG training data.
* ```preprocessed_eeg_test.npy```: the preprocessed EEG test data.

Both files are Python dictionaries with the following keys:
* ```preprocessed_eeg_data```: preprocessed EEG data in a 4-dimensional array of shape [16,540/200 train/test image conditions × 4/80 train/test EEG repetitions × 17 EEG channels × 100 EEG time points].
* ```ch_names```: names of the 17 occipital and parietal EEG channels retained during preprocessing.
* ```times```: the 100 time points of each EEG epoch (in seconds, with respect to stimulus onset).

In [ ]:
eeg_parent_dir = os.path.join(gdrive_data_parent_dir, 'eeg_dataset',
    'preprocessed_data', 'sub-01')
eeg_data_train = np.load(os.path.join(eeg_parent_dir,
    'preprocessed_eeg_training.npy'), allow_pickle=True).item()
eeg_data_test = np.load(os.path.join(eeg_parent_dir,
    'preprocessed_eeg_test.npy'), allow_pickle=True).item()

print('Training EEG data shape:')
print(eeg_data_train['preprocessed_eeg_data'].shape)
print('(Training image conditions × Training EEG repetitions × EEG channels × '
    'EEG time points)')

print('\nTest EEG data shape:')
print(eeg_data_test['preprocessed_eeg_data'].shape)
print('(Test image conditions × Test EEG repetitions × EEG channels × '
    'EEG time points)')

print('\nEEG channels:')
for c,chan in enumerate(eeg_data_train['ch_names']):
    print(c, chan)

print('\nEEG time points (in seconds):')
for t,time in enumerate(eeg_data_train['times']):
    print(t, np.round(time, decimals=2))

## 3. Visualize the preprocessed EEG data ERPs

Here you will plot the preprocessed EEG data event related potentials (ERPs) across time, for each EEG channel. ERPs consist in the EEG signal averaged across image conditions and repetitions.

The ERPs show peaks of activity every 200ms, consistent with the 200ms stimulus onset asynchronies (SOAs) of the rapid serial visual presentation (RSVP) paradigm used to collect the EEG data.

In [ ]:
erp_data_train = np.mean(eeg_data_train['preprocessed_eeg_data'], 1)
erp_data_test = np.mean(eeg_data_test['preprocessed_eeg_data'], 1)
erp_data_all = np.mean(np.append(erp_data_train, erp_data_test, 0), 0)

plt.figure()
plt.plot([-.2, .8], [0, 0], 'k--', [0, 0], [-1.5, 1.5], 'k--')
plt.plot(eeg_data_train['times'], np.transpose(erp_data_all));
plt.xlabel('Time (s)');
plt.xlim(left=-.2, right=.8)
plt.ylabel('Voltage');
plt.ylim(bottom=-1.5, top=1.5)
plt.title('EEG ERPs');

## 4. Load the images metadata

Next, you will load and visualize the metadata of the EEG responses' stimuli images. The images are divided into a training and a test partition (corresponding to the EEG training and test data partitions). The training partition has 1,654 object concepts, with 10 images per concept, for a total of 16,540 image conditions. The test partition has 200 object concepts with 1 image per concept, for a total of 200 image conditions.

In [ ]:
img_parent_dir  = os.path.join(gdrive_data_parent_dir, 'image_set')
img_metadata = np.load(os.path.join(img_parent_dir, 'image_metadata.npy'),
	allow_pickle=True).item()

n_train_img = len(img_metadata['train_img_concepts'])
n_train_concepts = len(np.unique(img_metadata['train_img_concepts']))
n_train_img_per_concept = int(n_train_img / n_train_concepts)
print('Training images: ' + str(n_train_img))
print('Image concepts: ' + str(n_train_concepts))
print('Images per concept: '+ str(n_train_img_per_concept))

n_test_img = len(img_metadata['test_img_concepts'])
n_test_concepts = len(np.unique(img_metadata['test_img_concepts']))
n_test_img_per_concept = int(n_test_img / n_test_concepts)
print('\nTest images: ' + str(n_test_img))
print('Image concepts: ' + str(n_test_concepts))
print('Images per concept: '+ str(n_test_img_per_concept))

## 5. Visualize the training images, and link them to the corresponding EEG responses

Here you can visualize arbitrary images from the **training** partition, and link them to their corresponding EEG responses. Select the image of interest by editing the ```train_img_idx``` variable with values in the range [0 16,539] (Python indexing stars from 0).

In [ ]:
train_img_idx =  0 #@param {type:"integer"}

eeg_data_single_image = eeg_data_train['preprocessed_eeg_data'][train_img_idx]
print('Training EEG single image data shape:')
print(eeg_data_single_image.shape)
print('(Training EEG repetitions × EEG channels × EEG time points)\n')

train_img_dir = os.path.join(img_parent_dir, 'training_images',
	img_metadata['train_img_concepts'][train_img_idx],
	img_metadata['train_img_files'][train_img_idx])
train_img = Image.open(train_img_dir).convert('RGB')

plt.figure()
plt.axis('off')
plt.imshow(train_img)
plt.title('Training image: ' + str(train_img_idx+1) + '\nImage concept: ' +\
	img_metadata['train_img_concepts'][train_img_idx] + '\nImage file: ' +\
	img_metadata['train_img_files'][train_img_idx]);

## 6. Visualize the test images, and link them to the corresponding EEG responses

Here you can visualize arbitrary images from the test partition, and link them to their corresponding EEG responses. Select the image of interest by editing the ```test_img_idx``` variable with values in the range [0 199] (Python indexing stars from 0).

In [ ]:
test_img_idx =  0 #@param {type:"integer"}

eeg_data_single_image = eeg_data_test['preprocessed_eeg_data'][train_img_idx]
print('Test EEG single image data shape:')
print(eeg_data_single_image.shape)
print('(Test EEG repetitions × EEG channels × EEG time points)\n')

test_img_dir = os.path.join(img_parent_dir, 'test_images',
	img_metadata['test_img_concepts'][test_img_idx],
	img_metadata['test_img_files'][test_img_idx])
test_img = Image.open(test_img_dir).convert('RGB')

plt.figure()
plt.axis('off')
plt.imshow(test_img)
plt.title('Test image: ' + str(test_img_idx+1) + '\nImage concept: ' +\
	img_metadata['test_img_concepts'][test_img_idx] + '\nImage file: ' +\
	img_metadata['test_img_files'][test_img_idx]);

## 7. Load the DNN feature maps

Now you will load the training an test images feature maps of a pretrained AlexNet architecture, that have been appended across all DNN layers and downsampled with principal component analysis (PCA). The DNN feature consists of two files:
* ```pca_feature_maps_training.npy```: the training DNN feature maps.
* ```pca_feature_maps_test.npy```: the test DNN feature maps.

Both files are Python dictionaries with the following key:
* ```all_layers```: DNN feature maps in a 2-dimensional array of shape [16,540/200 train/test image conditions × 3000 DNN feature maps PCA components].

In [ ]:
dnn_parent_dir = os.path.join(gdrive_data_parent_dir, 'dnn_feature_maps',
    'pca_feature_maps', 'alexnet', 'pretrained-True', 'layers-all')
dnn_fmaps_train = np.load(os.path.join(dnn_parent_dir,
    'pca_feature_maps_training.npy'), allow_pickle=True).item()
dnn_fmaps_test = np.load(os.path.join(dnn_parent_dir,
    'pca_feature_maps_test.npy'), allow_pickle=True).item()

print('Training DNN feature maps shape:')
print(dnn_fmaps_train['all_layers'].shape)
print('(Training image conditions × DNN feature maps PCA components)')

print('\nTraining DNN feature maps shape:')
print(dnn_fmaps_test['all_layers'].shape)
print('(Test image conditions × DNN feature maps PCA components)')

## 8. Train and test linearizing encoding models of EEG visual responses

In this last step you will train linearizing encoding models of EEG visual responses, that is, algorithms that predict the EEG responses to arbitrary stimuli images. Linearizing encoding models consist of linear regressions that map the DNN feature maps of certain image conditions onto the corresponding EEG responses. You will train independent linear regressions for each EEG feature (i.e., each combination of EEG channels and time points).

You will train linear regressions using the training DNN feature maps and training EEG data (averaged across repetitions), and use the learned regression weights to predict the EEG responses to the test images (using the test DNN feature maps). To reduce computational load, the training/test DNN feature maps are reduced to 100 PCA components.

Finally, you will quantify the encoding models prediction accuracy by correlating each predicted EEG feature with the corresponding biological/ground truth EEG feature (averaged across repetitions) across the 200 test image conditions, average the resulting correlation scores over channels, and plot them across time points.

In [ ]:
pred_eeg_data_test = np.zeros((len(eeg_data_test['preprocessed_eeg_data']),
    len(eeg_data_test['ch_names']),len(eeg_data_test['times'])))
for t in tqdm(range(len(eeg_data_test['times'])), desc='Encoding training'):
    for c in range(len(eeg_data_test['ch_names'])):
        reg = LinearRegression().fit(dnn_fmaps_train['all_layers'][:,:100],
            np.mean(eeg_data_train['preprocessed_eeg_data'][:,:,c,t], 1))
        pred_eeg_data_test[:,c,t] = reg.predict(
            dnn_fmaps_test['all_layers'][:,:100])

encoding_accuracy = np.zeros((len(eeg_data_test['ch_names']),
    len(eeg_data_test['times'])))
for t in range(len(eeg_data_test['times'])):
    for c in range(len(eeg_data_test['ch_names'])):
	    encoding_accuracy[c,t] = corr(pred_eeg_data_test[:,c,t],
            np.mean(eeg_data_test['preprocessed_eeg_data'][:,:,c,t], 1))[0]

plt.figure()
plt.plot([-.2, .8], [0, 0], 'k--', [0, 0], [-1, 1], 'k--')
plt.plot(eeg_data_test['times'], np.mean(encoding_accuracy, 0));
plt.xlabel('Time (s)');
plt.xlim(left=-.2, right=.8)
plt.ylabel('Pearson\'s $r$');
plt.ylim(bottom=-.05, top=.7)
plt.title('Encoding accuracy');